In [ ]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

Imports

In [ ]:
# External libraries
import os as so
import sys as s
import pathlib as pl
import torch
import torch.nn as nn
from torch import Tensor
from torch.utils.data import random_split
from torch.utils.data import DataLoader, ConcatDataset
import torcheval
from torcheval.metrics import MulticlassF1Score, Mean
import optuna as opt
import torchvision as tn
import sklearn as sn
from sklearn.metrics import f1_score
import pandas as ps
import numpy as ny
import typing as t
import pathlib as pl
import matplotlib.pyplot as pt
import random as rng
from tqdm import tqdm
import tqdm as tm
from pprint import pprint
from git import Repo

In [ ]:
# Add local package to path
if (p := pl.Path(so.getcwd(), '..').absolute().as_posix()) not in s.path:
    s.path.append(p)

# Local imports
from gic import *
from gic.data import GenImageDataset
from gic.tune import ClassificationTrainer
from gic.models.resnet import ResCNN, resnet_sampler
from gic.models.densenet import DenseCNN, densenet_sampler
from gic.models.convnext import ConvNextNet, convnext_sampler

HyperParameter Optimization

In [ ]:
# Find the best model using a trainer
trainer = ClassificationTrainer(
    seed=SEED,
    device=DEVICE,
    augment=False,
    model=DenseCNN,
    profiling=profiling,
    mix_float=mix_float,
    hps=densenet_sampler,
    dataset_path=DATA_PATH,
    num_workers=num_workers,
    prefetch_factor=prefetch_factor,
)

In [ ]:
# Search hyperparams
searcher = opt.samplers.TPESampler(n_startup_trials=5)
pruner = opt.pruners.NopPruner()
search = opt.create_study(direction='maximize', storage=db_uri, sampler=searcher, pruner=pruner)
search.optimize(trainer, n_trials=50, callbacks=[wn_callback])
wn.finish()

Final Training

In [ ]:
# Find the best model using a trainer
trainer = ClassificationTrainer(
    seed=SEED,
    device=DEVICE,
    augment=True,
    model=DenseCNN,
    profiling=profiling,
    mix_float=mix_float,
    hps=densenet_sampler,
    dataset_path=DATA_PATH,
    num_workers=num_workers,
    prefetch_factor=prefetch_factor,
)

In [ ]:
params = {
    'lr': 1e-3,
    'epochs': 150,
    'batch_size': 32,
    'optimizer': 'Adam',
    'weight_decay': 0.0000423,
    'dense': 128,
    'factor_c': 1,
    'features': 16,
    'factor_t': 1,
    'activ_fn': 'SiLU',
    'pool': 'max',
    'repeat': 3,
    'inner': 3,
    'f_drop': 0.2435,
    'c_drop': 0.1199,
}


# Train on `almost` the whole labeled dataset
trainer.train(params)
wn.finish()

Perform evaluation

In [ ]:
preds: t.List[int] = trainer.eval(params).tolist()
preds[:10]

In [ ]:
len(preds)

Save Submission

In [ ]:
test_data = GenImageDataset(DATA_PATH, 'test', True)
dataf = {'Image': [], 'Class': []}
dataf['Class'] = preds
test_data._GenImageDataset__data['Class'] = preds
test_data._GenImageDataset__data.to_csv(SUBMISSION_PATH, index=False)